# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('always')
%matplotlib inline
#%load_ext pycodestyle_magic

In [99]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=4, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return l_s * (1 - np.sum((l_c / l_s) ** 2, axis=1)[:, np.newaxis]) + \
               r_s * (1 - np.sum((r_c / r_s) ** 2, axis=1)[:, np.newaxis])
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(l_c * l_s * np.log(l_s)).sum(axis=1)[:, np.newaxis] - \
                (r_c * r_s * np.log(r_s)).sum(axis=1)[:, np.newaxis]

    def __misclass(self, l_c, l_s, r_c, r_s):
        return  (1 - np.max(l_s / l_c, axis=1))[:, np.newaxis] +  (r_c - np.max(r_s / r_c, axis=1))[:, np.newaxis]

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[k < np.sqrt(n_feature)]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[k < np.log(n_feature) / np.log(2)]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        #на вход подаётся столбец, соответствующий одному признаку (икс)
        #и классы ну вот этих объектов (игрек)
        #мы это, короче, сортируем
        #class_number - число классов
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        #Что делает этот блок кода?
        # Разбиваем классы на подмножества так, чтобы в каждом семействе было хотя 
        #бы min_samples_split классов
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        #Что делает этот блок кода?
        #переписываем разбиение через one hot code
        #class_increments - структура, которая переводит one hot code в требуемое
        #на следующем шаге
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        #выводит количества классов и число элемнтов одного класса в левой и правой кучке разбиения
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(sorted_y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        #считает impurities для всех разбиений и выбирает наименьшую
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmax(gs)
    
        # Что делает этот блок кода?
        #выдаёт параметры оптимального разбиения туда, откуда функция вызывается,
        #а именно: impurity и граница по тому параметру
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        print node_id
        self.tree[node_id] = dict()
        if (y.size < self.min_samples_split): 
            self.tree[node_id][0] = self.LEAF_TYPE #node type
            self.tree[node_id][1] = np.argmax(np.bincount(y)) #classification task answer
            self.tree[node_id][2] = np.bincount(y) / y.shape[0] #prob answer
        else:
                                                  
            self.tree[node_id][0] = self.NON_LEAF_TYPE;
                                                  
            #find out required feature
            max_gain = float('-inf')
            best_feature = 0
            best_threshold = 0.0
            for i in range(x.shape[1]):
                x_prop = x[:, i]
                gain, threshold = self.__find_threshold(x_prop, y)
                if(gain < max_gain):
                    max_gain = gain
                    best_feature = i
                    best_threshold = threshold
        
            self.tree[node_id]['feature_id'] = best_feature
            self.tree[node_id]['threshold'] = threshold
        
            #теперь пора сделать детишек
            x_l, x_r, y_l, y_r = self.__div_samples(x, y, best_feature, threshold)
            #if (y_l.size < self.min_samples_split) or (y_r.size < self.min_samples_split):
            #    self.tree[node_id][0] = self.LEAF_TYPE
            #    self.tree[node_id][1] = np.argmax(np.bincount(y)) #classification task answer
            #    self.tree[node_id][2] = np.bincount(y) / y.shape[0] #prob answer
            #else:
            self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1, best_feature)
            self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1, best_feature)
            pass
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.NON_LEAF_TYPE:
            feature_id = node['feature_id']
            threshold = node['threshold']
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.NON_LEAF_TYPE:
            feature_id = node['feature_id']
            threshold = node['threshold']
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [100]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [101]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [102]:
%time clf.fit(X_train, y_train)

CPU times: user 4.51 ms, sys: 29 µs, total: 4.54 ms
Wall time: 3.42 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [103]:
%time my_clf.fit(X_train, y_train)

0
1


ValueError: attempt to get argmax of an empty sequence

## Проверка качества работы на wine

In [75]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

1.0

In [76]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.18666666666666668

## Подготовка данных Speed Dating Data 

In [ ]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb

In [ ]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', 
                 encoding='cp1251')
df = df.iloc[:, :97]
#dropping
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o','met_o'], 
             axis=1)
df = df.drop(['undergra'], axis=1)


df = df.dropna(subset=['age'])

#field_cd??
df.loc[:, 'field_cd'] = df.loc[:,['field_cd']].fillna(0)
df = df.drop(['field'], axis=1)

#career
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)

df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)

df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)

#income
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])

#throwing away
df = df.drop(['sports','tvsports','exercise','dining','museums','art','hiking','gaming',
       'clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga'], axis=1)
df = df.drop(['expnum'], axis=1)

#attr1 correction
feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
(df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100


#attr2 correction
feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
(df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)

#mass dropping
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)


df = df.drop(['wave'], axis=1)

#male-female
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), 
                       on='pid', 
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                    random_state=123)
X

## Проверка скорости работы на Speed Dating Data 

In [40]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9407407407407408

In [41]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.18666666666666668

## Проверка качества работы на Speed Dating Data

In [ ]:
# тут должен быть код типа %time clf.fit(X_train, y_train)

In [ ]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)

## Задание 3

## Задание 4

## Задание 5

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=43)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)